DESeq analysis of DamID data, to see what sites are differentially methylated.

In [49]:
library(DESeq2)

Load the files, see what they look like

In [62]:
#This file is the AdRp file, with columns with counts less than 500000 removed, and with rows with rDNA or MtDNA removed
damid<-read.csv("AdRp_noMtDNA")
loc<-read.csv("Locations")
metadata<-read.csv("file_names")

In [51]:
head(loc)
length(loc)

,X,X0,X1,X2
1,0,211000022278049,390,394
2,1,211000022278049,537,541
3,2,211000022278049,573,577
4,3,211000022278049,837,841
5,4,211000022278049,1613,1617
6,5,211000022278049,1618,1622


[1] 4

have new locations list

In [52]:
loc=paste(loc$X0, ":", loc$X1, "-", loc$X2, sep="")

In [53]:
head(loc)

[1] "211000022278049:390-394"   "211000022278049:537-541"  
[3] "211000022278049:573-577"   "211000022278049:837-841"  
[5] "211000022278049:1613-1617" "211000022278049:1618-1622"

In [54]:
head(metadata)

,X,Rep,Tissue,Country,Line,NotSureProtein,protein
1,damID-Dam_Fr188_C1,1,C,Fr,188,damID,Dam
2,damID-Dam_Fr188_C2,2,C,Fr,188,damID,Dam
3,damID-Dam_Fr188_C3,3,C,Fr,188,damID,Dam
4,damID-Dam_Fr188_C4,4,C,Fr,188,damID,Dam
5,damID-Dam_Fr188_T3,3,T,Fr,188,damID,Dam
6,damID-Dam_Fr188_T4,4,T,Fr,188,damID,Dam


In [55]:
unique(metadata$Rep)
unique(metadata$Tissue)
unique(metadata$Country)
unique(metadata$Line)
unique(metadata$NotSureProtein) #this is batch
unique(metadata$protein)

[1]  1  2  3  4 NA

[1] C T W w
Levels: C T w W

[1] Fr Zi
Levels: Fr Zi

[1] 188 257 275  89 238

[1] damID  damID2 damID3
Levels: damID damID2 damID3

[1] Dam Lam D1 
Levels: D1 Dam Lam

What is line 275?  It's probably just a typo, and means 257.  There is already a replicate 1 for Dam Zi 257 though.  In doubt, we delete it.

In [63]:
metadata[metadata$Line == '275',]
metadata<-metadata[!(metadata$Line == '275'),]
unique(metadata$Line)

,X,Rep,Tissue,Country,Line,NotSureProtein,protein
11,damID-Dam_Zi275_T1,1,T,Zi,275,damID,Dam


[1] 188 257  89 238

In [64]:
#Replace the Na reps with 0
metadata[is.na(metadata)] <- 0
unique(metadata$Rep)

#Replace 'w' with 'W'
metadata$Tissue <- toupper(metadata$Tissue)
unique(metadata$Tissue)

#Replace metadata row names to conform to column names in damid (introduced in next cell)
rownames(metadata)=gsub('-', '.', metadata$X)
metadata$X = NULL

#Change to factors
metadata$Country<-factor(metadata$Country)
metadata$Line<-factor(metadata$Line)
metadata$Protein<-factor(metadata$protein)
metadata$Batch<-factor(metadata$NotSureProtein)

#Clean up names
metadata$protein<-NULL
metadata$NotSureProtein<-NULL

head(metadata)

[1] 1 2 3 4 0

[1] "C" "T" "W"

,Rep,Tissue,Country,Line,Protein,Batch
damID.Dam_Fr188_C1,1,C,Fr,188,Dam,damID
damID.Dam_Fr188_C2,2,C,Fr,188,Dam,damID
damID.Dam_Fr188_C3,3,C,Fr,188,Dam,damID
damID.Dam_Fr188_C4,4,C,Fr,188,Dam,damID
damID.Dam_Fr188_T3,3,T,Fr,188,Dam,damID
damID.Dam_Fr188_T4,4,T,Fr,188,Dam,damID


damid dataframe has no mitochondrial genome or rDNA, and we only use columns with sum of counts > 500000.

In [69]:
colnames(damid)
#there is a mix of uppercase and lower case W in the colnames, we make them all uppercase
colnames(damid)=gsub('w', 'W', colnames(damid))

#Remove the column with 275: 
damid$damID.Dam_Zi275_T1<-NULL

head(damid)
"dimensions of damid"
dim(damid)

[1] "X"                   "damID.Dam_Fr188_C1"  "damID.Dam_Fr188_C2" 
 [4] "damID.Dam_Fr188_C3"  "damID.Dam_Fr188_C4"  "damID.Dam_Fr188_T3" 
 [7] "damID.Dam_Fr188_T4"  "damID.Dam_Zi257_C1"  "damID.Dam_Zi257_C2" 
[10] "damID.Dam_Zi257_C3"  "damID.Dam_Zi257_W"   "damID.Lam_Fr188_C1" 
[13] "damID.Lam_Fr188_C2"  "damID.Lam_Fr188_C3"  "damID.Lam_Fr188_C"  
[16] "damID.Lam_Fr188_T2"  "damID.Lam_Fr188_T3"  "damID.Lam_Zi257_C1" 
[19] "damID.Lam_Zi257_C2"  "damID.Lam_Zi257_C3"  "damID.Lam_Zi257_T1" 
[22] "damID.Lam_Zi257_T2"  "damID.Lam_Zi257_T3"  "damID2.D1_Fr188_C3" 
[25] "damID2.D1_Fr188_T3"  "damID2.D1_Fr89_C2"   "damID2.D1_Fr89_C3"  
[28] "damID2.D1_Fr89_T3"   "damID2.D1_Zi238_C1"  "damID2.D1_Zi238_C3" 
[31] "damID2.D1_Zi238_T3"  "damID2.D1_Zi257_C1"  "damID2.D1_Zi257_C2" 
[34] "damID2.D1_Zi257_C3"  "damID2.D1_Zi257_T2"  "damID2.D1_Zi257_T3" 
[37] "damID2.Dam_Fr188_W"  "damID2.Dam_Fr89_C1"  "damID2.Dam_Fr89_C3" 
[40] "damID2.Dam_Fr89_T1"  "damID2.Dam_Fr89_T2"  "damID2.Dam_Zi238_C1"
[43] "damID2.Dam_Zi238_C2" "damID2.Dam_Zi238_C3" "damID2.Dam_Zi257_W" 
[46] "damID2.Lam_Fr89_C1"  "damID2.Lam_Fr89_C2"  "damID2.Lam_Zi238_C1"
[49] "damID2.Lam_Zi238_C2" "damID2.Lam_Zi238_C3" "damID2.Lam_Zi257_C4"
[52] "damID3.D1_Fr188_T"   "damID3.D1_Fr188_W1"  "damID3.D1_Fr188_W2" 
[55] "damID3.D1_Fr188_W3"  "damID3.D1_Fr89_W1"   "damID3.D1_Fr89_W2"  
[58] "damID3.D1_Fr89_W3"   "damID3.D1_Zi238_T"   "damID3.D1_Zi238_W1" 
[61] "damID3.D1_Zi238_W2"  "damID3.D1_Zi238_W3"  "damID3.D1_Zi257_T"  
[64] "damID3.D1_Zi257_W1"  "damID3.D1_Zi257_W2"  "damID3.D1_Zi257_W3" 
[67] "damID3.Dam_Fr188_T2" "damID3.Dam_Fr188_W1" "damID3.Dam_Fr188_W2"
[70] "damID3.Dam_Fr188_W3" "damID3.Dam_Fr89_W1"  "damID3.Dam_Fr89_W2" 
[73] "damID3.Dam_Fr89_W3"  "damID3.Dam_Zi238_T1" "damID3.Dam_Zi238_T2"
[76] "damID3.Dam_Zi238_W1" "damID3.Dam_Zi238_W2" "damID3.Dam_Zi238_W3"
[79] "damID3.Dam_Zi257_T1" "damID3.Dam_Zi257_T2" "damID3.Dam_Zi257_W1"
[82] "damID3.Dam_Zi257_W2" "damID3.Dam_Zi257_W3" "damID3.Lam_Fr188_T1"
[85] "damID3.Lam_Fr188_T4" "damID3.Lam_Fr188_W1" "damID3.Lam_Fr188_W2"
[88] "damID3.Lam_Fr188_W3" "damID3.Lam_Fr89_T1"  "damID3.Lam_Fr89_T2" 
[91] "damID3.Lam_Fr89_W1"  "damID3.Lam_Fr89_W2"  "damID3.Lam_Fr89_W3" 
[94] "damID3.Lam_Zi238_W1" "damID3.Lam_Zi238_W2" "damID3.Lam_Zi238_W3"
[97] "damID3.Lam_Zi257_W1" "damID3.Lam_Zi257_W2" "damID3.Lam_Zi257_W3"

,X,damID.Dam_Fr188_C1,damID.Dam_Fr188_C2,damID.Dam_Fr188_C3,damID.Dam_Fr188_C4,damID.Dam_Fr188_T3,damID.Dam_Fr188_T4,damID.Dam_Zi257_C1,damID.Dam_Zi257_C2,damID.Dam_Zi257_C3,ellip.h,damID3.Lam_Fr89_T2,damID3.Lam_Fr89_W1,damID3.Lam_Fr89_W2,damID3.Lam_Fr89_W3,damID3.Lam_Zi238_W1,damID3.Lam_Zi238_W2,damID3.Lam_Zi238_W3,damID3.Lam_Zi257_W1,damID3.Lam_Zi257_W2,damID3.Lam_Zi257_W3
1,0,3,1,0,0,3,3,1,2,0,⋯,1,0,1,2,1,0,0,0,1,0
2,1,3,0,0,0,2,3,1,1,0,⋯,1,0,0,1,0,1,0,1,0,0
3,2,3,0,0,0,2,3,1,1,0,⋯,1,0,0,1,0,1,1,1,0,0
4,3,1,0,0,0,2,1,2,1,1,⋯,1,0,2,0,0,1,1,1,0,0
5,4,0,2,2,2,0,1,1,2,0,⋯,1,1,1,1,0,0,1,0,1,2
6,5,0,2,2,2,0,1,1,2,0,⋯,1,1,1,1,0,0,1,0,1,2


[1] "dimensions of damid"

[1] 388464     99

### Extracting information for each line by protein by line combination in DESeq

We want to separate tissues, because that's what we did in our analysis of gene expression data.

The reason we run DESeq independently for each line is because we want the shrinkages used in the log2FoldChange calculation to be independent.  (Because of shrinkage, the log2foldchange might be more trustworthy than baseMean (ie. mean(D1)/mean(DAM) (or is it the inverse?)).

The following cell returns a list of files of the form:
paste("results_", tissue, "_", protein,"_", line, sep="")
which has all the information "baseMean","log2FoldChange","lfcSE","stat","pvalue","padj".

There are some combinations, like T 238 that can't be used for this analysis (no Lam or D1 entries for T 238, just Dam).


In [30]:
return_dataset_for_plot<-function(tissue,protein,line, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))
    colsLine<-c(grep(line, names(damid)))
    
    cols<-intersect(intersect(colsTissue,colsLine),union(colsDam,colsProtein))
    
    print(cols)

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam") & metadata$Line==line,]

    print(metadata)
    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein)
    suppressWarnings(dds<-DESeq(dataset))
    res.dds<-results(dds)
    write.csv(res.dds, file=paste("results_", tissue, "_", protein,"_", line, sep=""))
}


#There are some combinations, like T 238 that can't be used for this analysis (no Lam or D1 entries)
for (tissue in c("C", "W","T")) {
    for (line in c("238","257","89","188")) {
        for (protein in c("Lam", "D1")) {
            try(return_dataset_for_plot(tissue, protein, line, damid, metadata))
        }
    }
}


[1] 75 76
                    Rep Tissue Country Line Protein  Batch
damID3.Dam_Zi238_T1   1      T      Zi  238     Dam damID3
damID3.Dam_Zi238_T2   2      T      Zi  238     Dam damID3
[1] 32 60 75 76
                    Rep Tissue Country Line Protein  Batch
damID2.D1_Zi238_T3    3      T      Zi  238      D1 damID2
damID3.D1_Zi238_T     0      T      Zi  238      D1 damID3
damID3.Dam_Zi238_T1   1      T      Zi  238     Dam damID3
damID3.Dam_Zi238_T2   2      T      Zi  238     Dam damID3


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] 22 23 24 80 81
                    Rep Tissue Country Line Protein  Batch
damID.Lam_Zi257_T1    1      T      Zi  257     Lam  damID
damID.Lam_Zi257_T2    2      T      Zi  257     Lam  damID
damID.Lam_Zi257_T3    3      T      Zi  257     Lam  damID
damID3.Dam_Zi257_T1   1      T      Zi  257     Dam damID3
damID3.Dam_Zi257_T2   2      T      Zi  257     Dam damID3


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] 36 37 64 80 81
                    Rep Tissue Country Line Protein  Batch
damID2.D1_Zi257_T2    2      T      Zi  257      D1 damID2
damID2.D1_Zi257_T3    3      T      Zi  257      D1 damID2
damID3.D1_Zi257_T     0      T      Zi  257      D1 damID3
damID3.Dam_Zi257_T1   1      T      Zi  257     Dam damID3
damID3.Dam_Zi257_T2   2      T      Zi  257     Dam damID3


...

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1]  6  7 17 18 68 85 86
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Fr188_T3    3      T      Fr  188     Dam  damID
damID.Dam_Fr188_T4    4      T      Fr  188     Dam  damID
damID.Lam_Fr188_T2    2      T      Fr  188     Lam  damID
damID.Lam_Fr188_T3    3      T      Fr  188     Lam  damID
damID3.Dam_Fr188_T2   2      T      Fr  188     Dam damID3
damID3.Lam_Fr188_T1   1      T      Fr  188     Lam damID3
damID3.Lam_Fr188_T4   4      T      Fr  188     Lam damID3


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1]  6  7 26 53 68
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Fr188_T3    3      T      Fr  188     Dam  damID
damID.Dam_Fr188_T4    4      T      Fr  188     Dam  damID
damID2.D1_Fr188_T3    3      T      Fr  188      D1 damID2
damID3.D1_Fr188_T     0      T      Fr  188      D1 damID3
damID3.Dam_Fr188_T2   2      T      Fr  188     Dam damID3


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


## Extracting main effects info (also info about batch and interactions)

Here we don't separate information by line.  These give us general results on D1 and Lam, and tell us about the effect of Batch.  

The conclusion we get from doing a binomial test for carcass and D1: (full = ~Line*Protein; reduced= ~Line + Protein)  and (full = ~Line + Protein; reduced= ~Protein):
D1 protein binds to few places. We don't see a main effect for D1, so we also don’t predict to be able to see an interaction of line:protein (and we don't). However, the fact that there isn't a high percentage of LFC >0 and LFC <0  doesn’t mean there isn’t differential binding of D1. (Low counts= 83%).  If the (few) sites at which we see differential binding do  have differential gene expression by line, then this would point to a possible mechanism by which the Y chromosome can affect gene expression across the autosomes.  This is because we know that D1 binds to several Y chromosome satellites.  So this would be consistent with the hypothesis that the Y chromosome acts as a protein sink.
We can't test Batch for D1 because not enough variety of Batch for D1 protein.


The conclusion we get from doing a binomial test for carcass and Lam: (full = ~Line*Protein; reduced= ~Line + Protein) and (full = ~Line + Protein; reduced= ~Protein) and (full = ~Line*Protein + Batch; reduced= ~Line*Protein):
1. We see more binding of the Lam protein than D1 protein (this is expected).  Specifically, more main effects of D1, and we also see evidence for the interaction Line:protein.  We might be able to conclude from this that there is a link between variation in expression and variation in chromatin state (once we do that analysis), but unlike in the D1 case we can't draw conclusions about the mechanism.  We can simply conclude that some underlying genetic mechanism on Y chromosome affects both changes in expression and changes in chromatin.  It could be for example Y chromo affects expression which in turn affects chromatin state, or Y chromo affects chromatin state which in turn affects gene expression.  It could also be structural (different sites are near the nuclear envelope depending on the size of the Y chromosome).
2.  Batch doesn't seem to have much of an effect, at least in the carcass and Lam case.

Whar do we make of the fact that we dont see diff expression for W?



Here we test interactions

In [42]:
return_dataset_for_plot2_interaction<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line + Line:Protein)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~Line + Protein)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    #write.csv(res.dds, file=paste("results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    for (protein in c("Lam", "D1")) {
        print(c(tissue, protein))
        #try(return_dataset_for_plot2_interaction(tissue, protein, line, damid, metadata))
        return_dataset_for_plot2_interaction(tissue, protein, damid, metadata)
    }
}


[1] "C"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 24281, 6.3% 
LFC < 0 (down)   : 28593, 7.4% 
outliers [1]     : 887, 0.23% 
low counts [2]   : 120500, 31% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "C"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388463 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2485, 0.64% 
LFC < 0 (down)   : 1628, 0.42% 
outliers [1]     : 14, 0.0036% 
low counts [2]   : 323851, 83% 
(mean count < 5)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"   "Lam"


...

[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 68, 0.018% 
LFC < 0 (down)   : 42, 0.011% 
outliers [1]     : 458, 0.12% 
low counts [2]   : 75315, 19% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 13, 0.0033% 
LFC < 0 (down)   : 8, 0.0021% 
outliers [1]     : 431, 0.11% 
low counts [2]   : 248499, 64% 
(mean count < 5)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"   "Lam"


factor levels were dropped which had no samples


ERROR: Error in checkFullRank(modelMatrix): the model matrix is not full rank, so the model cannot be fit as specified.
  Levels or combinations of levels without any samples have resulted in
  column(s) of zeros in the model matrix.

  Please read the vignette section 'Model matrix not full rank':

  vignette('DESeq2')


In [75]:
#Same as above (we handled the mistake, which was that there was a single line == 275 in our dataset)
#Also, there are no 238/Lam columns, so the matrix model is not full rank. 
#For now, we just consider T and D1

return_dataset_for_plot2_interaction<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))
    print(dim(damid[,cols]))
    
    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]
    print(metadata)
    print(dim(metadata))
    
    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line + Line:Protein)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~Line + Protein)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    #write.csv(res.dds, file=paste("results_", tissue, "_", protein, sep=""))
}

for (tissue in c("T")) {
    for (protein in c("D1")) {
        print(c(tissue, protein))
        #try(return_dataset_for_plot2_interaction(tissue, protein, line, damid, metadata))
        return_dataset_for_plot2_interaction(tissue, protein, damid, metadata)
    }
}





[1] "T"  "D1"
[1] 388464     17
                    Rep Tissue Country Line Protein  Batch
damID.Dam_Fr188_T3    3      T      Fr  188     Dam  damID
damID.Dam_Fr188_T4    4      T      Fr  188     Dam  damID
damID2.D1_Fr188_T3    3      T      Fr  188      D1 damID2
damID2.D1_Fr89_T3     3      T      Fr   89      D1 damID2
damID2.D1_Zi238_T3    3      T      Zi  238      D1 damID2
damID2.D1_Zi257_T2    2      T      Zi  257      D1 damID2
damID2.D1_Zi257_T3    3      T      Zi  257      D1 damID2
damID2.Dam_Fr89_T1    1      T      Fr   89     Dam damID2
damID2.Dam_Fr89_T2    2      T      Fr   89     Dam damID2
damID3.D1_Fr188_T     0      T      Fr  188      D1 damID3
damID3.D1_Zi238_T     0      T      Zi  238      D1 damID3
damID3.D1_Zi257_T     0      T      Zi  257      D1 damID3
damID3.Dam_Fr188_T2   2      T      Fr  188     Dam damID3
damID3.Dam_Zi238_T1   1      T      Zi  238     Dam damID3
damID3.Dam_Zi238_T2   2      T      Zi  238     Dam damID3
damID3.Dam_Zi257_T1   1 

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388359 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1888, 0.49% 
LFC < 0 (down)   : 1527, 0.39% 
outliers [1]     : 2485, 0.64% 
low counts [2]   : 368648, 95% 
(mean count < 13)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



Here we test main effects

In [43]:
return_dataset_for_plot2_mainEffect<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line)
    suppressWarnings(dds<-DESeq(dataset))

    ddsLRT<-nbinomLRT(dds, reduced= ~Line)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    #write.csv(res.dds, file=paste("results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    for (protein in c("Lam", "D1")) {
        print(c(tissue, protein))
        #try(return_dataset_for_plot2_interaction(tissue, protein, line, damid, metadata))
        return_dataset_for_plot2_mainEffect(tissue, protein, damid, metadata)
    }
}



[1] "C"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 72152, 19% 
LFC < 0 (down)   : 65362, 17% 
outliers [1]     : 3574, 0.92% 
low counts [2]   : 37656, 9.7% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "C"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388463 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2789, 0.72% 
LFC < 0 (down)   : 4433, 1.1% 
outliers [1]     : 68, 0.018% 
low counts [2]   : 248531, 64% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "W"   "Lam"


...

[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 19550, 5% 
LFC < 0 (down)   : 20629, 5.3% 
outliers [1]     : 673, 0.17% 
low counts [2]   : 105440, 27% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388456 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 56168, 14% 
LFC < 0 (down)   : 62316, 16% 
outliers [1]     : 1282, 0.33% 
low counts [2]   : 67780, 17% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388404 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 141, 0.036% 
LFC < 0 (down)   : 195, 0.05% 
outliers [1]     : 6433, 1.7% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



Testing batch

In [77]:
#C/D1 might not have enough diversity in batch to test batch effect

return_dataset_for_plot2_batch<-function(tissue,protein, damid, metadata){
    colsTissue<-c(grep(paste("_", tissue, sep=''), names(damid)))
    colsProtein<-c(grep(paste(protein, "_",sep=''), names(damid)))
    colsDam<-c(grep("Dam_", names(damid)))

    cols<-intersect(colsTissue,union(colsDam,colsProtein))

    metadata<-metadata[metadata$Tissue==tissue & (metadata$Protein==protein | metadata$Protein=="Dam"),]

    dataset <- DESeqDataSetFromMatrix(countData = damid[,cols], colData = metadata, design = ~Protein + Line + Line:Protein + Batch)
    suppressWarnings(dds<-DESeq(dataset))
    
    ddsLRT<-nbinomLRT(dds, reduced= ~Line + Protein + Line:Protein)
    res.ddsLRT<-results(ddsLRT)

    print("summary LRT")
    summary(res.ddsLRT)
    #write.csv(res.dds, file=paste("results_", tissue, "_", protein, sep=""))
}

for (tissue in c("C", "W","T")) {
    for (protein in c("Lam", "D1")) {
        print(c(tissue, protein))
        #try(return_dataset_for_plot2_interaction(tissue, protein, line, damid, metadata))
        try(return_dataset_for_plot2_batch(tissue, protein, damid, metadata))
    }
}



[1] "C"   "Lam"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
found results columns, replacing these


[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1688, 0.43% 
LFC < 0 (down)   : 6433, 1.7% 
outliers [1]     : 703, 0.18% 
low counts [2]   : 218409, 56% 
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "C"  "D1"


factor levels were dropped which had no samples


[1] "W"   "Lam"


...

[1] "summary LRT"

out of 388464 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 41357, 11% 
LFC < 0 (down)   : 35821, 9.2% 
outliers [1]     : 0, 0% 
low counts [2]   : 135566, 35% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] "T"   "Lam"


factor levels were dropped which had no samples


[1] "T"  "D1"


factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
633 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest
found results columns, replacing these
633 rows did not converge in beta, labelled in mcols(object)$fullBetaConv. Use larger maxit argument with nbinomLRT


[1] "summary LRT"

out of 388359 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1901, 0.49% 
LFC < 0 (down)   : 5077, 1.3% 
outliers [1]     : 0, 0% 
low counts [2]   : 323760, 83% 
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



find the regions with differential expression.  correlate to gene expression regions